
# Was tut dieses Skript?

Dieses Skript 
berechnet die Mercator-Kacheln (Tiles) für jedes deutsche Bundesland
die Ergebnisse als JSON und als Plot.
Dieses Skript muss nur ein einiges Mal ausgeführt werden, um die Kacheln zu bestimmen.


In [1]:
import geopandas as gpd
import mercantile
from shapely.geometry import box, Polygon, MultiPolygon
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import json


In [2]:
## TODO: # Nur ausführen, wenn entsprechende JSON-Datei nicht existiert

In [3]:
def get_bbox_from_polygon(polygon):
    """Return bounding box (minx, miny, maxx, maxy) of a polygon."""
    return polygon.bounds


def get_tiles_from_bbox(bbox, zoom=14):
    """Return mercantile tile list for a bbox at a given zoom."""
    minx, miny, maxx, maxy = bbox
    return list(mercantile.tiles(minx, miny, maxx, maxy, zooms=[zoom]))


def filter_tiles_by_polygon(tiles, polygon):
    """Return only those tiles whose centroid lies inside the polygon."""
    filtered = []
    for tile in tqdm(tiles, desc="Filtering tiles"):
        bounds = mercantile.bounds(tile)
        tile_box = box(bounds.west, bounds.south, bounds.east, bounds.north)
        centroid = tile_box.centroid
        if polygon.intersects(centroid):
            filtered.append(tile)
    return filtered


def filter_tiles_by_polygon_with_buffer(tiles, polygon, buffer_km=2):
    """
    Filtert Tiles nach Polygon mit Buffer.
    Gibt Liste von Tuples zurück: (tile, overlap_area)
    
    Args:
        tiles: Liste von Mercantile Tiles
        polygon: Shapely Polygon (in EPSG:4326)
        buffer_km: Buffer-Distanz in Kilometern
    
    Returns:
        Liste von (tile, overlap_area) Tuples
    """
    # Projiziere Polygon zu Meter-CRS (EPSG:25832 für Deutschland)
    polygon_gdf = gpd.GeoDataFrame(geometry=[polygon], crs="EPSG:4326")
    polygon_proj = polygon_gdf.to_crs("EPSG:25832").geometry.iloc[0]
    
    # Erstelle Buffer (in Metern)
    buffered_polygon = polygon_proj.buffer(buffer_km * 1000)
    
    # Zurück zu EPSG:4326 für Tile-Vergleich
    buffered_gdf = gpd.GeoDataFrame(geometry=[buffered_polygon], crs="EPSG:25832")
    buffered_polygon_wgs84 = buffered_gdf.to_crs("EPSG:4326").geometry.iloc[0]
    
    filtered = []
    for tile in tqdm(tiles, desc="Filtering tiles with buffer"):
        bounds = mercantile.bounds(tile)
        tile_box = box(bounds.west, bounds.south, bounds.east, bounds.north)
        
        # Prüfe ob Tile den buffered Polygon schneidet
        if buffered_polygon_wgs84.intersects(tile_box):
            # Berechne Überlappungsfläche (in EPSG:25832 für korrekte Flächenberechnung)
            tile_gdf = gpd.GeoDataFrame(geometry=[tile_box], crs="EPSG:4326")
            tile_proj = tile_gdf.to_crs("EPSG:25832").geometry.iloc[0]
            
            overlap = polygon_proj.intersection(tile_proj)
            overlap_area = overlap.area if not overlap.is_empty else 0
            
            filtered.append((tile, overlap_area))
    
    return filtered


def assign_tiles_to_bundeslaender(bundeslaender_data, buffer_km=2):
    """
    Weist jedes Tile dem Bundesland mit der größten Überlappung zu.
    
    Args:
        bundeslaender_data: Liste von Dicts mit 'id', 'name', 'polygon'
        buffer_km: Buffer-Distanz in Kilometern
    
    Returns:
        Dict: {bundesland_id: [(tile, overlap_area), ...]}
    """
    # Sammle alle Tiles mit ihren Überlappungen pro Bundesland
    all_tiles_by_bundesland = {}
    
    for bl_data in bundeslaender_data:
        bl_id = bl_data['id']
        polygon = bl_data['polygon']
        
        bbox = get_bbox_from_polygon(polygon)
        tiles = get_tiles_from_bbox(bbox, zoom=14)
        tiles_with_overlap = filter_tiles_by_polygon_with_buffer(tiles, polygon, buffer_km)
        
        all_tiles_by_bundesland[bl_id] = tiles_with_overlap
    
    # Erstelle Mapping: tile -> (bundesland_id, overlap_area)
    tile_to_bundesland = {}
    
    for bl_id, tiles_with_overlap in all_tiles_by_bundesland.items():
        for tile, overlap_area in tiles_with_overlap:
            tile_key = (tile.x, tile.y, tile.z)
            
            if tile_key not in tile_to_bundesland:
                tile_to_bundesland[tile_key] = (bl_id, overlap_area)
            else:
                # Prüfe ob dieses Bundesland größere Überlappung hat
                existing_bl_id, existing_overlap = tile_to_bundesland[tile_key]
                if overlap_area > existing_overlap:
                    tile_to_bundesland[tile_key] = (bl_id, overlap_area)
    
    # Gruppiere Tiles zurück nach Bundesland
    final_tiles_by_bundesland = {bl_id: [] for bl_id in all_tiles_by_bundesland.keys()}
    
    for tile_key, (bl_id, overlap_area) in tile_to_bundesland.items():
        x, y, z = tile_key
        tile = mercantile.Tile(x=x, y=y, z=z)
        final_tiles_by_bundesland[bl_id].append(tile)
    
    return final_tiles_by_bundesland


def plot_tiles_and_boundary(filtered_tiles, boundary_polygon, bundesland_name="Region", save_path=None, output_folder="output", show=False):
    """Plot the filtered tiles and boundary together using projected CRS."""
    # Erstelle GeoDataFrame aus Tiles
    tile_polygons = []
    for tile in filtered_tiles:
        bounds = mercantile.bounds(tile)
        tile_polygons.append(Polygon([
            (bounds.west, bounds.south),
            (bounds.west, bounds.north),
            (bounds.east, bounds.north),
            (bounds.east, bounds.south)
        ]))

    tile_gdf = gpd.GeoDataFrame(geometry=tile_polygons, crs="EPSG:4326")
    boundary_gdf = gpd.GeoDataFrame(geometry=[boundary_polygon], crs="EPSG:4326")

    # Projiziere beides z. B. nach EPSG:3857
    tile_gdf_proj = tile_gdf.to_crs("EPSG:3857")
    boundary_gdf_proj = boundary_gdf.to_crs("EPSG:3857")

    # Plot
    fig, ax = plt.subplots(figsize=(8, 8))
    tile_gdf_proj.plot(ax=ax, color='green', edgecolor='black', alpha=0.5)
    boundary_gdf_proj.plot(ax=ax, color='blue', alpha=0.3, edgecolor='black')

    ax.set_axis_off()
    ax.set_title(f"{bundesland_name} – Tile Coverage ({len(filtered_tiles)} tiles)")

    if save_path is not None:
        os.makedirs(output_folder, exist_ok=True)
        full_path = os.path.join(output_folder, save_path)
        plt.savefig(full_path, bbox_inches='tight', pad_inches=0.1)
        print(f"✔️ Plot saved to: {full_path}")
    
    if show:
        plt.show()
    
    plt.close(fig)  # 🧼 Wichtig: Figure schließen, um Speicher zu sparen


def save_tiles_to_json(tiles, bundesland_id, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    tile_list = [{"x": t.x, "y": t.y, "z": t.z} for t in tiles]
    path = os.path.join(output_folder, f"{bundesland_id}_tiles.json")
    with open(path, "w") as f:
        json.dump(tile_list, f)
    print(f"✔️ Tiles saved to: {path}")

In [4]:
# Lade Bundesländer (mit Caching)
bundesland_geojson_url = "https://raw.githubusercontent.com/isellsoap/deutschlandGeoJSON/main/2_bundeslaender/1_sehr_hoch.geo.json"
bundesland_cache_path = os.path.join("prep", "bundeslaender.geojson")

# Lade aus Cache oder downloade neu
if os.path.exists(bundesland_cache_path):
    print(f"📂 Lade Bundesländer aus Cache: {bundesland_cache_path}")
    bland = gpd.read_file(bundesland_cache_path)
else:
    print(f"📥 Lade Bundesländer von: {bundesland_geojson_url}")
    bland = gpd.read_file(bundesland_geojson_url)
    
    # Speichere in Cache
    os.makedirs("prep", exist_ok=True)
    bland.to_file(bundesland_cache_path, driver="GeoJSON")
    print(f"💾 Bundesländer gespeichert in: {bundesland_cache_path}")

📥 Lade Bundesländer von: https://raw.githubusercontent.com/isellsoap/deutschlandGeoJSON/main/2_bundeslaender/1_sehr_hoch.geo.json
💾 Bundesländer gespeichert in: prep/bundeslaender.geojson


In [ ]:


# Bereite Bundesland-Daten vor
bundeslaender_data = []
for _, row in bland.iterrows():
    bundeslaender_data.append({
        'id': row["id"],
        'name': row["name"],
        'polygon': row["geometry"]
    })

print(f"📍 Verarbeite {len(bundeslaender_data)} Bundesländer mit 2km Buffer")
print(f"⚙️  Tiles werden dem Bundesland mit größter Überlappung zugeordnet")
print(f"{'='*70}\n")

# Weist Tiles eindeutig zu (verhindert Duplikate)
BUFFER_KM = 2
tiles_by_bundesland = assign_tiles_to_bundeslaender(bundeslaender_data, buffer_km=BUFFER_KM)

# # Speichere und plotte Ergebnisse
# for bl_data in bundeslaender_data:
#     b_id = bl_data['id']
#     name = bl_data['name']
#     polygon = bl_data['polygon']
#     filtered_tiles = tiles_by_bundesland[b_id]
    
#     print(f"{b_id} – {name}: {len(filtered_tiles)} tiles")
    
#     save_tiles_to_json(filtered_tiles, b_id, output_folder="/output")
    
#     plot_tiles_and_boundary(
#         filtered_tiles,
#         polygon,
#         bundesland_name=name,
#         save_path=f"{b_id}_tiles.png",
#         output_folder="/output",
#         show=False
#     )

# print(f"\n{'='*70}")
# print(f"✅ Fertig! Alle Bundesländer verarbeitet mit {BUFFER_KM}km Buffer")
# print(f"{'='*70}")

📍 Verarbeite 16 Bundesländer mit 2km Buffer
⚙️  Tiles werden dem Bundesland mit größter Überlappung zugeordnet



Filtering tiles with buffer: 100%|██████████| 13568/13568 [00:10<00:00, 1343.01it/s]


DE-BW – Baden-Württemberg: 14160 tiles


PermissionError: [Errno 13] Permission denied: '/output'

In [7]:
# Speichere und plotte Ergebnisse
for bl_data in bundeslaender_data:
    b_id = bl_data['id']
    name = bl_data['name']
    polygon = bl_data['polygon']
    filtered_tiles = tiles_by_bundesland[b_id]
    
    print(f"{b_id} – {name}: {len(filtered_tiles)} tiles")
    
    save_tiles_to_json(filtered_tiles, b_id, output_folder="output/tile_cache/")
    
    plot_tiles_and_boundary(
        filtered_tiles,
        polygon,
        bundesland_name=name,
        save_path=f"{b_id}_tiles.png",
        output_folder="output/tile_cache/",
        show=False
    )

print(f"\n{'='*70}")
print(f"✅ Fertig! Alle Bundesländer verarbeitet mit {BUFFER_KM}km Buffer")
print(f"{'='*70}")

DE-BW – Baden-Württemberg: 14160 tiles
✔️ Tiles saved to: output/tile_cache/DE-BW_tiles.json
✔️ Plot saved to: output/tile_cache/DE-BW_tiles.png
DE-BY – Bayern: 28376 tiles
✔️ Tiles saved to: output/tile_cache/DE-BY_tiles.json
✔️ Plot saved to: output/tile_cache/DE-BY_tiles.png
DE-BE – Berlin: 399 tiles
✔️ Tiles saved to: output/tile_cache/DE-BE_tiles.json
✔️ Plot saved to: output/tile_cache/DE-BE_tiles.png
DE-BB – Brandenburg: 13585 tiles
✔️ Tiles saved to: output/tile_cache/DE-BB_tiles.json
✔️ Plot saved to: output/tile_cache/DE-BB_tiles.png
DE-HB – Bremen: 145 tiles
✔️ Tiles saved to: output/tile_cache/DE-HB_tiles.json
✔️ Plot saved to: output/tile_cache/DE-HB_tiles.png
DE-HH – Hamburg: 361 tiles
✔️ Tiles saved to: output/tile_cache/DE-HH_tiles.json
✔️ Plot saved to: output/tile_cache/DE-HH_tiles.png
DE-HE – Hessen: 8704 tiles
✔️ Tiles saved to: output/tile_cache/DE-HE_tiles.json
✔️ Plot saved to: output/tile_cache/DE-HE_tiles.png
DE-MV – Mecklenburg-Vorpommern: 12294 tiles
✔️ Tiles